In [1]:
import pandas as pd 
import numpy as np

In [5]:
df=pd.read_csv(r"C:\Users\HP\Downloads\E-commerce Return Rate Reduction Analysis.csv")

In [7]:
df

,Order ID,Customer ID,Product Category,Region,Marketing Channel,Order Date,Return Date,Return Reason,Returned
0,1001,C001,Electronics,North,Email,2024-07-01,2024-07-10,Defective,Yes
1,1002,C002,Clothing,South,Social Media,2024-07-02,NaN,Too Small,No
2,1003,C003,Home Decor,West,Search Engine,2024-07-03,2024-07-15,Not as Described,Yes
3,1004,C004,Clothing,East,Affiliate,2024-07-04,2024-07-12,Too Large,Yes
4,1005,C005,Sports,North,Email,2024-07-05,NaN,Changed Mind,No
5,1006,C006,Electronics,West,Social Media,2024-07-06,2024-07-20,Defective,Yes
6,1007,C007,Home Decor,South,Search Engine,2024-07-07,NaN,Not as Described,No
7,1008,C008,Clothing,North,Email,2024-07-08,2024-07-18,Poor Quality,Yes
8,1009,C009,Sports,East,Social Media,2024-07-09,NaN,Changed Mind,No
9,1010,C010,Electronics,South,Affiliate,2024-07-10,2024-07-22,Wrong Item Sent,Yes


In [9]:
# Convert dates to datetime format
df['Order Date'] = pd.to_datetime(df['Order Date'])
df['Return Date'] = pd.to_datetime(df['Return Date'], errors='coerce')

# Standardize 'Returned' column
df['Returned'] = df['Returned'].map({'Yes': 1, 'No': 0})

# Fill missing values if needed (e.g., Return Reason for non-returns)
df['Return Reason'] = df['Return Reason'].fillna('No Return')

# Final check
print(df.info())
print(df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Order ID           10 non-null     int64         
 1   Customer ID        10 non-null     object        
 2   Product Category   10 non-null     object        
 3   Region             10 non-null     object        
 4   Marketing Channel  10 non-null     object        
 5   Order Date         10 non-null     datetime64[ns]
 6   Return Date        6 non-null      datetime64[ns]
 7   Return Reason      10 non-null     object        
 8   Returned           10 non-null     int64         
dtypes: datetime64[ns](2), int64(2), object(5)
memory usage: 852.0+ bytes
None
   Order ID Customer ID Product Category Region Marketing Channel Order Date  \
0      1001        C001      Electronics  North             Email 2024-07-01   
1      1002        C002         Clothing  South      So

In [11]:
# Return rate per product category
category_returns = df.groupby("Product Category")["Returned"].agg(['count', 'sum'])
category_returns['Return %'] = 100 * category_returns['sum'] / category_returns['count']
print(category_returns)


                  count  sum    Return %
Product Category                        
Clothing              3    2   66.666667
Electronics           3    3  100.000000
Home Decor            2    1   50.000000
Sports                2    0    0.000000


In [18]:
import random

# Assign random suppliers
df['Supplier'] = df['Product Category'].map({
    'Electronics': random.choice(['TechWorld', 'ElecPro']),
    'Clothing': random.choice(['FashionHub', 'WearIt']),
    'Home Decor': random.choice(['HomeStyle', 'DecorNation']),
    'Sports': random.choice(['Sporty', 'FitLife'])
})

In [20]:
df

,Order ID,Customer ID,Product Category,Region,Marketing Channel,Order Date,Return Date,Return Reason,Returned,Supplier
0,1001,C001,Electronics,North,Email,2024-07-01,2024-07-10,Defective,1,TechWorld
1,1002,C002,Clothing,South,Social Media,2024-07-02,NaT,Too Small,0,WearIt
2,1003,C003,Home Decor,West,Search Engine,2024-07-03,2024-07-15,Not as Described,1,DecorNation
3,1004,C004,Clothing,East,Affiliate,2024-07-04,2024-07-12,Too Large,1,WearIt
4,1005,C005,Sports,North,Email,2024-07-05,NaT,Changed Mind,0,FitLife
5,1006,C006,Electronics,West,Social Media,2024-07-06,2024-07-20,Defective,1,TechWorld
6,1007,C007,Home Decor,South,Search Engine,2024-07-07,NaT,Not as Described,0,DecorNation
7,1008,C008,Clothing,North,Email,2024-07-08,2024-07-18,Poor Quality,1,WearIt
8,1009,C009,Sports,East,Social Media,2024-07-09,NaT,Changed Mind,0,FitLife
9,1010,C010,Electronics,South,Affiliate,2024-07-10,2024-07-22,Wrong Item Sent,1,TechWorld


In [39]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import classification_report

# Select features
features = ['Product Category', 'Region', 'Marketing Channel']
X = df[features]
y = df['Returned']

# One-hot encoding
X_encoded = pd.get_dummies(X)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

# Logistic regression
model = LogisticRegression()
model.fit(X_train, y_train)

# Predict and evaluate
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred, zero_division=0))

# Predict return probability for each record
df['Return Probability'] = model.predict_proba(X_encoded)[:, 1]


              precision    recall  f1-score   support

           0       0.00      0.00      0.00       2.0
           1       0.00      0.00      0.00       0.0

    accuracy                           0.00       2.0
   macro avg       0.00      0.00      0.00       2.0
weighted avg       0.00      0.00      0.00       2.0

